In [1]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np

In [29]:
## Acessando pelo pacote da BD

df=bd.read_sql("SELECT ano, tema, sigla_uf,policia_civil_efetivo_homem,policia_civil_efetivo_mulher,policia_civil_efetivo_total FROM `basedosdados-projetos.republica.indicadores_estadic_gerais` where ano=2019 and tema='seguranca_publica' order by sigla_uf", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 27/27 [00:00<00:00, 86.29rows/s]


In [ ]:
## Utilizando como fonte para população a projeção populacional do IBGE em 2019 (data da pesquisa)

In [8]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Documentos auxiliares')

In [9]:
pop = pd.read_csv("pop_projetada_2019_ibge.csv", sep=";")

In [22]:
pop

,nome_uf,sigla_uf,pop. Projetada
0,Rondônia,RO,1777225
1,Acre,AC,881935
2,Amazonas,AM,4144597
3,Roraima,RR,605761
4,Pará,PA,8602865
5,Amapá,AP,845731
6,Tocantins,TO,1572866
7,Maranhão,MA,7075181
8,Piauí,PI,3273227
9,Ceará,CE,9132078


In [10]:
pop['pop. Projetada']= pop['pop. Projetada'].str.replace('.','')

In [11]:
pop['pop. Projetada']= pd.to_numeric(pop['pop. Projetada'])

In [21]:
pop.columns

Index(['ano', 'nome_uf', 'sigla_uf', 'pop_projetada'], dtype='object')

In [12]:
pop['ano']=2019

In [13]:
pop=pop[['ano','nome_uf', 'sigla_uf', 'pop. Projetada']]

In [14]:
pop=pop.rename(columns={'pop. Projetada':'pop_projetada'})

In [8]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('Datalake')


In [22]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de implementação da legislação.'),
 bigquery.SchemaField('nome_uf','STRING',description='Nome da UF.'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('pop_projetada','INTEGER',description='Sigla da Unidade da Federação.')
 ]

In [23]:
table_ref = dataset_ref.table('IBGE_projecao_populacional')

In [24]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [25]:
job = client.load_table_from_dataframe(pop, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=6fb963d8-30f6-4245-bce3-43adb8086a0a>

In [ ]:
## TRazendo do datalake

In [30]:
### Join com o dado de policial

df1 = df.merge(pop[['sigla_uf', 'pop_projetada']],right_on='sigla_uf', left_on='sigla_uf')

In [31]:
df1

,ano,tema,sigla_uf,policia_civil_efetivo_homem,policia_civil_efetivo_mulher,policia_civil_efetivo_total,pop_projetada
0,2019,seguranca_publica,AC,531,242,773,881935
1,2019,seguranca_publica,AL,1563,400,1963,3337357
2,2019,seguranca_publica,AM,1601,878,2479,4144597
3,2019,seguranca_publica,AP,584,256,840,845731
4,2019,seguranca_publica,BA,4810,2383,7193,14873064
5,2019,seguranca_publica,CE,2724,940,3664,9132078
6,2019,seguranca_publica,DF,3182,1266,4448,3015268
7,2019,seguranca_publica,ES,1544,614,2158,4018650
8,2019,seguranca_publica,GO,2936,1522,4458,7018354
9,2019,seguranca_publica,MA,1652,398,2050,7075181


In [32]:
df1['taxa'] = df1['policia_civil_efetivo_total']/df1['pop_projetada']*1000

In [21]:
df1

,ano,tema,sigla_uf,policia_civil_efetivo_total,pop_projetada,taxa
0,2019,seguranca_publica,AC,773,881935,8.764818
1,2019,seguranca_publica,AL,1963,3337357,5.8819
2,2019,seguranca_publica,AM,2479,4144597,5.981281
3,2019,seguranca_publica,AP,840,845731,9.932236
4,2019,seguranca_publica,BA,7193,14873064,4.83626
5,2019,seguranca_publica,CE,3664,9132078,4.01223
6,2019,seguranca_publica,DF,4448,3015268,14.751591
7,2019,seguranca_publica,ES,2158,4018650,5.369963
8,2019,seguranca_publica,GO,4458,7018354,6.351917
9,2019,seguranca_publica,MA,2050,7075181,2.897452


Subindo o arquivo para o gbq

In [23]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')


In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ano                           27 non-null     Int64  
 1   tema                          27 non-null     object 
 2   sigla_uf                      27 non-null     object 
 3   policia_civil_efetivo_homem   27 non-null     Int64  
 4   policia_civil_efetivo_mulher  27 non-null     Int64  
 5   policia_civil_efetivo_total   27 non-null     Int64  
 6   pop_projetada                 27 non-null     int64  
 7   taxa                          27 non-null     Float64
dtypes: Float64(1), Int64(4), int64(1), object(2)
memory usage: 2.0+ KB


In [35]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de implementação da legislação.'),
 bigquery.SchemaField('tema','STRING',description='Tema de origem daquele dado na ESTADIC'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
bigquery.SchemaField('policia_civil_efetivo_homem','INTEGER',description='Quantitativo total de homens policiais civis.'),
bigquery.SchemaField('policia_civil_efetivo_mulher','INTEGER',description='Quantitativo total de mulheres policiais civis.'),
 bigquery.SchemaField('policia_civil_efetivo_total','INTEGER',description='Quantitativo total de efetivo de policiais civis.'),
 bigquery.SchemaField('pop_projetada','INTEGER',description='População projetada pelo IBGE para o ano de referência'),
 bigquery.SchemaField('taxa','FLOAT',description='taxa de policiais por 1000 habitantes')
 
 ]

In [36]:
table_ref = dataset_ref.table('ESTADIC_policial_civil_habitantes')

In [38]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [39]:
#para excluir tabelas
client.delete_table('perfil_remuneracao.ESTADIC_policial_civil_habitantes', not_found_ok=True)

In [40]:
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=adea101e-d195-4dff-bf81-9cbde92cb2c6>